In [3]:
!pip install fasttext
#https://fasttext.cc/docs/en/unsupervised-tutorial.html

In [2]:
!pip install numpy==1.23.5


In [4]:
!pip install names-dataset

#https://pypi.org/project/names-dataset/

In [5]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import accuracy_score, classification_report

import fasttext
import re
import pandas as pd
import numpy as np

from names_dataset  import NameDataset



In [6]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\sa-zA-Z0-9@\[\]]',' ',text) # Удаляет пунктуацию
    text = re.sub(r'\w*\d+\w*', '', text) # Удаляет цифры
    text = re.sub('\s{2,}', " ", text) # Удаляет ненужные пробелы
    return text


In [7]:
df = pd.read_csv('surname-nationality.csv')
df

,rank,surname,nationality,frequency,census_count
0,1.0,Tesfaye,Ethiopian,0.011905,1167260
1,2.0,Mohammed,Ethiopian,0.011111,1084839
2,3.0,Getachew,Ethiopian,0.009174,895366
3,4.0,Abebe,Ethiopian,0.008475,825501
4,5.0,Girma,Ethiopian,0.008403,822765
...,...,...,...,...,...
36236,NaN,Yun,Chinese,NaN,1
36237,NaN,Zhai,Chinese,NaN,1
36238,NaN,Zhi,Chinese,NaN,1
36239,NaN,Zhuan,Chinese,NaN,1


In [8]:
df['nationality'].unique()

array(['Ethiopian', 'Honduran', 'Nigerian', 'Malaysian', 'Chilean',
       'Portuguese', 'Papua New Guinean', 'Algerian', 'Brazilian',
       'Venezuelan', 'Ukrainian', 'South African', 'Nicaraguan',
       'Moroccan', 'Finnish', 'Mexican', 'Palestinian', 'Nepalese',
       'Peruvian', 'Dutch', 'Arabic', 'Irish', 'Spanish', 'French',
       'German', 'English', 'Korean', 'Indian', 'Vietnamese', 'Scottish',
       'Japanese', 'Polish', 'Greek', 'Czech', 'Italian', 'Russian',
       'Chinese'], dtype=object)

In [9]:
arr = df['nationality'].unique()
arr.size

37

In [10]:
df['surname'] = df['surname'].apply(clean_text)

In [11]:
df = df.replace(['Ethiopian', 'Honduran', 'Nigerian', 'Malaysian', 'Chilean',
       'Portuguese', 'Papua New Guinean', 'Algerian', 'Brazilian',
       'Venezuelan', 'South African', 'Nicaraguan',
       'Moroccan', 'Finnish', 'Mexican', 'Palestinian', 'Nepalese',
       'Peruvian', 'Dutch', 'Arabic', 'Irish', 'Spanish', 'French',
       'German', 'English', 'Korean', 'Indian', 'Vietnamese', 'Scottish',
       'Japanese', 'Polish', 'Greek', 'Czech', 'Italian',
       'Chinese'], [2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2])
df = df.replace(['Ukrainian', 'Russian'],[1,1])
df

/tmp/ipython-input-11-3704480269.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(['Ukrainian', 'Russian'],[1,1])


,rank,surname,nationality,frequency,census_count
0,1.0,tesfaye,2,0.011905,1167260
1,2.0,mohammed,2,0.011111,1084839
2,3.0,getachew,2,0.009174,895366
3,4.0,abebe,2,0.008475,825501
4,5.0,girma,2,0.008403,822765
...,...,...,...,...,...
36236,NaN,yun,2,NaN,1
36237,NaN,zhai,2,NaN,1
36238,NaN,zhi,2,NaN,1
36239,NaN,zhuan,2,NaN,1


In [12]:
df_copy = df.copy()

#russian_rows = df_copy[df_copy['nationality'] == 'Russian']

# Рассчитайте количество строк для удаления (60% от количества строк с 'Russian')
#rows_to_remove = int(0.6 * len(russian_rows))

# Получите индексы случайных строк для удаления
#rows_to_delete = np.random.choice(russian_rows.index, size=rows_to_remove, replace=False)

# Удалите выбранные строки из копии датафрейма
#df_copy.drop(rows_to_delete, inplace=True)

In [13]:
russian_rows = df.loc[df['nationality'] == 1]

# Выведите результат на экран
russian_rows.head(100)


,rank,surname,nationality,frequency,census_count
8055,1.0,melnyk,1,0.003106,141260
8056,2.0,shevchenko,1,0.002950,134372
8057,3.0,bondarenko,1,0.002571,116925
8058,4.0,kovalenko,1,0.002551,116023
8059,5.0,boiko,1,0.002375,108189
...,...,...,...,...,...
8150,96.0,serhyenko,1,0.000591,26882
8151,97.0,kornyenko,1,0.000590,26856
8152,98.0,martynenko,1,0.000587,26704
8153,99.0,velychko,1,0.000586,26674


In [14]:
df_copy.columns

Index(['rank', 'surname', 'nationality', 'frequency', 'census_count'], dtype='object')

In [15]:
df_shuffled = df_copy.sample(frac=1, random_state=42)
df_shuffled

,rank,surname,nationality,frequency,census_count
35709,NaN,achteren,2,NaN,1
4265,937.0,cossio,2,0.000088,1554
13781,765.0,haboush,2,0.000287,1307
5557,504.0,bouabdallah,2,0.000284,10966
31626,NaN,mahovikov,1,NaN,1
...,...,...,...,...,...
16850,834.0,hoogendijk,2,0.000140,2363
6265,212.0,albuquerque,2,0.000459,98223
11284,269.0,varis,2,0.000552,3034
860,533.0,parada,2,0.000162,1426


# Data splitting


In [16]:
train, test = train_test_split(df_shuffled, test_size = 0.2)

In [17]:
train

,rank,surname,nationality,frequency,census_count
9848,794.0,makhubo,2,0.000161,8719
19032,NaN,cartwright,2,NaN,1
18136,NaN,gunther,2,NaN,2
27769,NaN,baklund,1,NaN,1
3348,20.0,castillo,2,0.006098,107334
...,...,...,...,...,...
659,332.0,archaga,2,0.000450,3968
9467,413.0,brits,2,0.000321,17362
18080,NaN,frank,2,NaN,1
14992,976.0,baraghare,2,0.000047,1346


# TXT creation

In [18]:
with open('train.txt', 'w') as f:
    for each_text, each_label in zip(train['surname'], train['nationality']):
        f.writelines(f'__label__{each_label} {each_text}\n')

In [19]:
with open('test.txt', 'w') as f:
    for each_text, each_label in zip(test['surname'], test['nationality']):
        f.writelines(f'__label__{each_label} {each_text}\n')

In [20]:
!head -n 10 train.txt


__label__2 makhubo
__label__2 cartwright
__label__2 gunther
__label__1 baklund
__label__2 castillo
__label__2 abou omra
__label__2 piirainen
__label__1 awduloff
__label__1 agapeev
__label__2 satake


# Model building


In [21]:
model1 = fasttext.train_supervised('train.txt', lr=1.0, epoch=10, minn=2, maxn=9, dim=300)


In [22]:
def print_results(sample_size, precision, recall):
    precision   = round(precision, 2)
    recall      = round(recall, 2)
    print(f'{sample_size=}')
    print(f'{precision=}')
    print(f'{recall=}')

# Применяем функцию
print_results(*model1.test('test.txt'))


sample_size=7249
precision=0.96
recall=0.96


In [23]:
model1.predict('jhons')

(('__label__2',), array([0.99958891]))

In [24]:
def model_pred(name):
  output = model1.predict(name)
  pred = output[0][0]
  if pred == '__label__1':
    pred = 'Russian'
  else: pred = 'Not Russian'
  return pred

In [25]:
print(model_pred('jhons'))

Not Russian


# Prediction with another dataset

In [26]:
nd = NameDataset()


In [27]:
def Last_name_country(data):
  last_name_data = data['last_name']['country']
  most_popular_country = max(last_name_data, key=last_name_data.get)
  if most_popular_country == 'Russian Federation':
    most_popular_country = 'Russia'
  return most_popular_country

def First_name_country(data, first_name):
  first_name_data = data['first_name']['country']
  most_popular_country = max(first_name_data, key=first_name_data.get)
  if most_popular_country == 'Russian Federation':
    most_popular_country = 'Russia'
  return most_popular_country


In [28]:
name = 'Alexander Glukhov'
name = name.lower()
Fname = name.split()[0]
Lname = name.split()[1]
data_last_name = nd.search(Lname)
data_first_name = nd.search(Fname)

if (data_first_name['first_name'] is  None):
  F_name_country = 'NaN'
else:
  F_name_country = First_name_country(data_first_name, Fname)

#F_name_country = First_name_country(data_first_name)

if (data_last_name['last_name'] is  None):
  print('model:')
  L_name_country = model_pred(Lname)
else:
  L_name_country = Last_name_country(data_last_name)

print('Name Country:', F_name_country)
print('Surname Country:', L_name_country)

if (Fname == 'Andrew' or Fname == 'Alex' or Fname == 'Alexander'):
  if L_name_country == 'Russian':
    print('Result: Russia')
  else:print('Result: Not Russia')
else:
  if L_name_country == 'Russian' or F_name_country == 'Russian' or L_name_country == 'Russia' or F_name_country == 'Russia':
    print('Result: Russia')
  else: print('Result: Not Russia')



Name Country: Colombia
Surname Country: Russia
Result: Russia
